# Exp T2-spinodal: Measuring λ↓ (Collapse → Ordered Recovery Threshold)

## 目的
Collapse状態からλを段階的に下げ、回復が起きる閾値（λ↓）を測定。

## 実験設計
- **Phase 1 (Init)**: λ=0.70で50ep → Collapse状態確立
- **Phase 2 (Sweep)**: λを段階的に下降
  - 0.54 → 0.52 → 0.50 → 0.48 → 0.46 → 0.44 → 0.42 → 0.40 → 0.38
  - 各λで10ep保持
- **回復判定**: error ≤ 0.20 が3回連続 → 停止、λ↓を記録
- **η**: 0.4
- **Seeds**: 10（スクリーニング）

## 科学的目標
- λ↓の分布を測定
- T1-spinodal（λ↑）と組み合わせてヒステリシス幅を定量化
- **ヒステリシス幅 = λ↑ − λ↓**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_T_spinodal'
NOTEBOOK_ID = 'T2-spinodal'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (λ↓ measurement, η=0.4)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Spinodal experiment specific (COLLAPSE init)
NOISE_RATE = 0.4
INIT_LAMBDA = 0.70           # High λ for collapse init
INIT_EPOCHS = 50
INIT_COLLAPSE_THRESHOLD = 0.35  # Must be in collapsed state (error > 0.35)

# λ sweep parameters (downward for λ↓)
LAMBDA_SWEEP = [0.54, 0.52, 0.50, 0.48, 0.46, 0.44, 0.42, 0.40, 0.38]
EPOCHS_PER_LAMBDA = 10
RECOVERY_THRESHOLD = 0.20  # error ≤ this = recovered/ordered
RECOVERY_PERSIST = 3       # consecutive epochs to confirm

N_SEEDS = 10  # Screening first

print(f'η = {NOISE_RATE}')
print(f'Init: λ={INIT_LAMBDA} for {INIT_EPOCHS}ep (collapse state)')
print(f'Sweep: {LAMBDA_SWEEP}')
print(f'Per λ: {EPOCHS_PER_LAMBDA}ep')
print(f'Recovery: error≤{RECOVERY_THRESHOLD} for {RECOVERY_PERSIST} consecutive')
print(f'Seeds: {N_SEEDS}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    if sched is not None:
        sched.step()
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_spinodal_down(seed, train_loader, test_loader, clean_labels, noisy_labels):
    """Run spinodal experiment: collapse init → sweep λ downward → find recovery point"""
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed)
    model = get_resnet18().to(device)
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    
    # Phase 1: Initialize to COLLAPSE state (high λ)
    print(f'    Phase 1: Init COLLAPSE (λ={INIT_LAMBDA}, {INIT_EPOCHS}ep)...')
    for ep in range(INIT_EPOCHS):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, INIT_LAMBDA, cached_gv_ref)
        if (ep + 1) % 10 == 0:
            err = 1 - evaluate(model, test_loader)
            trajectory.append({'phase': 'init', 'lambda': INIT_LAMBDA, 'epoch': ep+1, 'error': err})
            print(f'      Ep {ep+1}: err={err:.4f}')
    
    init_error = 1 - evaluate(model, test_loader)
    
    # Check if in collapsed state (error should be HIGH)
    if init_error < INIT_COLLAPSE_THRESHOLD:
        print(f'    ⚠️ Init failed: {init_error:.4f} < {INIT_COLLAPSE_THRESHOLD} (not collapsed!)')
        return {
            'seed': seed, 'init_error': init_error, 'init_success': False,
            'lambda_down': None, 'recovered': False, 'trajectory': trajectory
        }
    
    print(f'    ✅ Init OK (collapsed): {init_error:.4f}')
    print(f'    Phase 2: λ sweep downward...')
    
    # Phase 2: Sweep λ downward
    lambda_down = None
    recovery_count = 0
    total_ep = INIT_EPOCHS
    
    for lam in LAMBDA_SWEEP:
        print(f'      λ={lam:.2f}: ', end='')
        
        for ep in range(EPOCHS_PER_LAMBDA):
            train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref)
            total_ep += 1
            err = 1 - evaluate(model, test_loader)
            trajectory.append({'phase': 'sweep', 'lambda': lam, 'epoch': total_ep, 'error': err})
            
            # Check recovery
            if err <= RECOVERY_THRESHOLD:
                recovery_count += 1
                if recovery_count >= RECOVERY_PERSIST:
                    lambda_down = lam
                    print(f'✅ RECOVERED at ep {total_ep} (err={err:.4f})')
                    break
            else:
                recovery_count = 0
        
        if lambda_down is not None:
            break
        
        final_err = 1 - evaluate(model, test_loader)
        print(f'err={final_err:.4f} (still collapsed)')
    
    if lambda_down is None:
        print(f'    ⏱️ No recovery down to λ={LAMBDA_SWEEP[-1]}')
    
    return {
        'seed': seed,
        'init_error': init_error,
        'init_success': True,
        'lambda_down': lambda_down,
        'recovered': lambda_down is not None,
        'final_lambda': lam,
        'final_error': 1 - evaluate(model, test_loader),
        'trajectory': trajectory
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_seeds = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done_seeds = {r['seed'] for r in results}
    print(f'Loaded: {len(done_seeds)} done')

for seed in range(N_SEEDS):
    if seed in done_seeds:
        continue
    
    run_num = len(results) + 1
    print(f'\n{"="*60}')
    print(f'[{run_num}/{N_SEEDS}] Seed {seed} | {NOTEBOOK_ID}')
    print(f'{"="*60}')
    
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_spinodal_down(seed, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{seed:03d}'
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_seeds.add(seed)
    
    # Summary
    if result['recovered']:
        status = f"✅ λ↓={result['lambda_down']:.2f}"
    else:
        status = f"⏱️ No recovery (down to λ={result['final_lambda']:.2f})"
    print(f'\n  {status} | {dt/60:.1f}min')
    
    # Stats
    recovered_results = [r for r in results if r.get('recovered', False)]
    if recovered_results:
        lambdas = [r['lambda_down'] for r in recovered_results]
        print(f'  λ↓ stats: n={len(lambdas)}, mean={np.mean(lambdas):.3f}, std={np.std(lambdas):.3f}')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*60 + f'\n{NOTEBOOK_ID} DONE\n' + '='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'trajectory'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY | η={NOISE_RATE}')
print('='*60)

n_total = len(df)
n_recovered = df['recovered'].sum()
print(f'\n📊 Results: {n_recovered}/{n_total} recovered ({100*n_recovered/n_total:.0f}%)')

if n_recovered > 0:
    lambda_downs = df[df['recovered'] == True]['lambda_down'].values
    print(f'\n📈 λ↓ Distribution:')
    print(f'   Mean: {lambda_downs.mean():.3f}')
    print(f'   Std:  {lambda_downs.std():.3f}')
    print(f'   Min:  {lambda_downs.min():.2f}')
    print(f'   Max:  {lambda_downs.max():.2f}')
else:
    print('\n⚠️ No recovery events observed!')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. λ↓ histogram
ax = axes[0]
recovered_df = df[df['recovered'] == True]
if len(recovered_df) > 0:
    ax.hist(recovered_df['lambda_down'], bins=np.arange(0.36, 0.56, 0.02), 
            color='blue', alpha=0.7, edgecolor='black')
    ax.axvline(recovered_df['lambda_down'].mean(), color='darkblue', linestyle='--', 
               linewidth=2, label=f"Mean λ↓={recovered_df['lambda_down'].mean():.3f}")
ax.set_xlabel('λ↓ (Recovery Threshold)', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('λ↓ Distribution (Collapse → Ordered)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Sample trajectories
ax = axes[1]
for r in results[:5]:
    if r['trajectory']:
        sweep_traj = [t for t in r['trajectory'] if t['phase'] == 'sweep']
        if sweep_traj:
            eps = [t['epoch'] for t in sweep_traj]
            errs = [t['error'] for t in sweep_traj]
            color = 'blue' if r['recovered'] else 'red'
            ax.plot(eps, errs, alpha=0.7, color=color, linewidth=1.5,
                   label=f"seed {r['seed']}: λ↓={r.get('lambda_down', 'N/A')}")

ax.axhline(RECOVERY_THRESHOLD, color='green', linestyle='--', linewidth=2, label=f'Recovery ({RECOVERY_THRESHOLD})')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Error Trajectories During λ Sweep', fontsize=12)
ax.legend(loc='upper right', fontsize=8)
ax.grid(True, alpha=0.3)

# 3. Error vs λ
ax = axes[2]
for r in results[:5]:
    if r['trajectory']:
        sweep_traj = [t for t in r['trajectory'] if t['phase'] == 'sweep']
        if sweep_traj:
            lambdas = [t['lambda'] for t in sweep_traj]
            errs = [t['error'] for t in sweep_traj]
            color = 'blue' if r['recovered'] else 'red'
            ax.scatter(lambdas, errs, alpha=0.5, color=color, s=20)

ax.axhline(RECOVERY_THRESHOLD, color='green', linestyle='--', linewidth=2)
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Error vs λ', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_analysis.png', dpi=150)
plt.show()

print(f'\n📈 Figure saved')

In [ ]:
# Hysteresis width calculation (if T1-spinodal results available)
print('='*60)
print('HYSTERESIS WIDTH CALCULATION')
print('='*60)

# Try to load T1-spinodal results
t1_results_path = f'{SAVE_DIR}/T1-spinodal_results.json'
if os.path.exists(t1_results_path):
    t1_results = json.load(open(t1_results_path))
    t1_collapsed = [r for r in t1_results if r.get('collapsed', False)]
    
    if t1_collapsed and n_recovered > 0:
        lambda_up_mean = np.mean([r['lambda_up'] for r in t1_collapsed])
        lambda_down_mean = lambda_downs.mean()
        hysteresis_width = lambda_up_mean - lambda_down_mean
        
        print(f'\n📊 λ↑ (ordered→collapse): {lambda_up_mean:.3f}')
        print(f'📊 λ↓ (collapse→ordered): {lambda_down_mean:.3f}')
        print(f'\n🔄 HYSTERESIS WIDTH = {hysteresis_width:.3f}')
    else:
        print('\n⚠️ Insufficient data for hysteresis calculation')
else:
    print(f'\n⚠️ T1-spinodal results not found at {t1_results_path}')
    print('   Run T1-spinodal first to calculate hysteresis width')